In [1]:
import pandas 
import opensoundscape

In [1]:
# utilities that should probably go in another module
# generalized cross-correlation
def gcc(x, y, max_delay_samples=None, filter='phat', epsilon = 0.01):
    """
    GCC implementation based on Knapp and Carter - code adapted from
    github.com/axeber01/ngcc
    Args:
        x: 1d numpy array of audio samples
        y: 1d numpy array of audio samples
        max_delay_samples: maximum possible delay between the 2 signals in max_delay_samples
        filter: which filter to use in the gcc.
            'phat' - Phase transform,
            'roth',
            'scot' - Smoothed Coherence Transform,
            'ht' - Hannan and Thomson
        epsilon = used to ensure denominator is non-zero.
    """
    n = x.shape[0] + y.shape[0]

    # Generalized Cross Correlation Phase Transform
    X = np.fft.rfft(x, n=n)
    Y = np.fft.rfft(y, n=n)
    Gxy = X * np.conj(Y)

    if filter == 'phat':
        phi = 1 / (np.abs(Gxy) + epsilon)

    elif filter == 'roth':
        phi = 1 / (X * torch.conj(X) + epsilon)

    elif filter == 'scot':
        Gxx = X * np.conj(X)
        Gyy = Y * np.conj(Y)
        phi = 1 / (np.sqrt(X * Y) + epsilon)

    elif filter == 'ht':
        Gxx = X * np.conj(X)
        Gyy = Y * np.conj(Y)
        gamma = Gxy / np.sqrt(Gxx * Gxy)
        phi = np.abs(gamma)**2 / (np.abs(Gxy)
                                     * (1 - gamma)**2 + epsilon)
    elif filter == 'cc':
        phi = 1.0
    else:
        raise ValueError("Unsupported filter. Must be one of: 'ht', 'phat', 'roth','scot'")


    #set the max delay in number of samples
    if max_delay_samples:
        max_delay_samples = np.minimum(max_delay_samples, int(n/2))
    else:
        max_delay_samples = int(n/2)

    cc = np.fft.irfft(Gxy * phi, n)

    return cc

def gillette_localize(
        receivers=list,
        delays=list,
        temp=20,
        m = [0],
        exact=True,
        summary=False,
        confint=False,
        alpha=0.05,
        td_error=False,
        total_td_error=False
    ):
        """
        Calculate the estimated location of a sound's source using the
        algorithm laid out in Gillette and Silverman (2008)
        Args:
            receivers: A numpy array of coordinates for microphones used to
            record the sound. The number of microphones needed should
            be two more than the dimensions being localized in. The
            first row will be treated as a reference point for the
            algorithm.
            tdoa: A list of time delays. Each entry should be the time
            delay for the corresponding item in the receivers list
            (i.e. the first item is the delay for the first receiver).
            The first item in this list should be 0, with all other
            entries centered around that.
            temp: ambient temperature in Celsius. Defaults to 20.
            exact: computes an exact solution if True, computes estimates
            with uncertainty if false. Defaults to True
            summary: displays a summary of the estimates if True. Defaults
            to false.
            confint: outputs confidence intervals for the estimated
            coordinates if true. Defaults to false.
            alpha: Determines confidence level of the confidence intervals.
            Defaults to 0.05.
            m: the index of the reference mic. Defaults to 0.
            td_error: Computes the expected time delay from the estimated
            source location, centered around the reference mic, for each
            microphone.
            total_td_error: Computes the euclidean norm of the errors
            provided by td_error.
        Returns:
            an array with the estimated coordinates and the estimated
            distance from the reference mic. (One reference mic and two
            additional mics, this is a 2 item array containing an estima
            -ted x coordinate and a distance.)
        """
        import opensoundscape.localization as loc
        import statsmodels.api as sm
        C = loc.calc_speed_of_sound(temperature=20)
        # Compile know receiver locations and distance delays into an output vector
        out_knowns = []
        in_knowns = np.zeros(((len(receivers) - len(m))*len(m), 2 + len(m)))
        toa = np.array(delays)
        r = 0
        out_knowns = []
        for k in range(len(m)):
            tdoa = toa - toa[m[k]] # Use the speed of sound to convert time delays to "distance delays"
            diffs = []
            for delay in tdoa:
                diffs.append(float(delay * loc.calc_speed_of_sound(20)))
            for i in range(len(receivers)):
                        if i in m:
                            continue
                        else:
                            w = diffs[i] ** 2
                            for j in range(len(receivers[i])):
                                w = w - receivers[i][j] ** 2 + receivers[m[k]][j] ** 2
                            w = w / 2
                            out_knowns.append(w)
            for i in range(len(receivers)):
                    if i in m:
                        continue
                    else:
                        q = 0
                        for j in range(len(receivers[i])):
                            z = receivers[m[k]][j] - receivers[i][j]
                            in_knowns[r][q] = z
                            q += 1
                        in_knowns[r][q+k] = diffs[i]
                        r += 1
                        continue

            # Using least squares, compute the final estimated location of source
        location = sm.OLS(out_knowns, in_knowns).fit()
        return location.params, location.summary(alpha = alpha), location.conf_int(alpha = alpha)
        if summary == True:
            return location.summary()


In [1]:
# make a class that we will use to contain a model object, list of files and thresholds
# this will be called Localizer
# we will use this class for localizing sound sources from synchronized audio files
class Localizer:
    def __init__(self, model, files, aru_coords, sample_rate, min_number_of_arus, thresholds = None, predictions = None, bandpass_range = None, max_delay = None, cc_threshold = 0):
        # initialize the class
        # model is a trained opensoundscape model
        # files is a list of synchronized audio files
        # aru_coords is a dictionary of aru coordinates, with key aru file path, and value (x,y) coordinates
        # thresholds is a dictionary of thresholds for each class
        # predictions is a pandas dataframe of predictions
        self.model = model
        self.files = files
        self.aru_coords = aru_coords
        self.thresholds = thresholds
        self.SAMPLE_RATE = sample_rate
        self.min_number_of_arus = min_number_of_arus
        self.predictions = predictions
        self.bandpass_range = bandpass_range
        self.max_delay = max_delay
        self.cc_threshold = cc_threshold

        # initialize the below intermediates as None. #TODO: work out how to do this correctly
        self.detections = None
        self.cross_correlations = None
        self.filtered_cross_correlations = None
    
    def get_predictions(self):
        # get CNN predictions from synchronized audio files
        # return a pandas dataframe with the results
        if self.predictions is None:
            self.predictions = self.model.predict(self.files, activation_layer = None)
        else:
            raise UserWarning("Predictions already exist - set predictions to None if you want to re-run predictions")
        return self.predictions

    def threshold_predictions(self):
        # use a set of thresholds to filter the predictions
        if self.predictions is None:
            print("No predictions exist - running predictions")
            self.get_predictions()
        all_sp_detections = []
        for species in self.predictions.columns:
            df = predictions_df.loc[:,[species]] #must be a dataframe
            detections = Localizer._get_detections(df, cnn_score_threshold = self.thresholds[species])
            grouped_detections = Localizer._group_detections(detections, self.aru_coords, self.max_distance_between_recorders)
            all_sp_detections.append(grouped_detections)
        detections_df = pandas.concat(all_sp_detections, axis=1)
        self.detections = detections_df
        return detections_df

    def cross_correlate(self):
        # cross correlate the predictions
        # return a pandas dataframe with the results      
        if self.bandpass_range is None:
            raise UserWarning("No bandpass range specified")
        if self.max_delay is None:
            raise UserWarning("No max delay specified")
        if self.detections is None:
            print("No detections exist - running threshold_predictions")
            self.threshold_predictions()
        # get the cross-correlations
        all_ccs = []
        all_tds = []
        for index, row in self.detections.iterrows():
            cc,td = Localizer._get_cross_correlations(
                reference_file=row["reference_file"],
                other_files=row["other_files"],
                start_time=row["time"][0],
                end_time=row["time"][1],
                bandpass_range=self.bandpass_ranges[species],
                max_delay=self.max_delay,
                SAMPLE_RATE=44100,
                )
            all_ccs.append(cc)
            all_tds.append(td)
        self.cross_correlations = self.detections.copy()
        self.cross_correlations["cross_correlations"] = all_ccs
        self.cross_correlations["time_delays"] = all_tds
        return self.cross_correlations
    
    def filter_cross_correlations(self):
        # filter the cross correlations
        # return a pandas dataframe with the results
        if self.cross_correlations is None:
            print("No cross correlations exist - running cross_correlate")
            self.cross_correlate()
        # filter the cross-correlations 
        above_threshold = [cc > self.cc_threshold for cc in self.cross_correlations["cross_correlations"]]
    
        n_before = len(self.cross_correlations) # number of rows before filtering

        filtered_ccs = []
        filtered_files = []
        filtered_tdoas = []
        for i in range(len(self.cross_correlations)):
            mask = above_threshold[i]
            cc = self.cross_correlations["cross_correlations"].iloc[i]
            other_files = np.array(self.cross_correlations["other_files"].iloc[i])
            tdoa = np.array(self.cross_correlations["time_delays"].iloc[i])

            filtered_ccs.append(cc[mask])
            filtered_files.append(other_files[mask])
            filtered_tdoas.append(tdoa[mask])
        
        filtered_cross_correlations = self.cross_correlations.copy()

        filtered_cross_correlations["cross_correlations"] = filtered_ccs
        filtered_cross_correlations["other_files"] = filtered_files
        filtered_cross_correlations["time_delays"] = filtered_tdoas

        # Filter by the cc scores. If less than min_number_of_ARUs have cc_score above threshold, drop them.
        ccs = [np.array(scores) for scores in filtered_cross_correlations["cross_correlations"]]
        num_ccs_above_threshold = [sum(a > self.cc_threshold) for a in ccs]
        mask = np.array(num_ccs_above_threshold) >= self.min_number_of_ARUs - 1
        filtered_cross_correlations = filtered_cross_correlations[mask]

        n_after = len(filtered_cross_correlations) # number of rows after filtering
        print(f"{n_before - n_after} rows deleted")
        self.filtered_cross_correlations = filtered_cross_correlations
        return filtered_cross_correlations


    def localize(algorithm = "gillette"):
        # localize the detections
        # return a pandas dataframe with the results
        #TODO: make work for 3d 

        localized = self.filtered_cross_correlations.copy()
        locations = []
        if  self.filtered_cross_correlations is None:
            print("No filtered cross_correlations exist - running filter_cross_correlations")
            self.filter_cross_correlations()
        if algorithm == "gillette":
            # localize using gillette
            
            for index, row in filtered_detections.iterrows():
                reference = row["reference_file"]
                others = row["other_files"]
                reference_coords = aru_coords_df.loc[reference]
                others_coords = [aru_coords_df.loc[i] for i in others]
                all_coords = [reference_coords] + others_coords
                # add 0 tdoa for reference receiver
                delays = np.insert(row["time_delays"], 0, 0)

            location,_,_=gillette_localize(all_coords, delays)
            locations.append(location)
            filtered_detections["predicted_x"] = [locations[i][0] for i in range(len(locations))]
            filtered_detections["predicted_y"] = [locations[i][1] for i in range(len(locations))]
        elif algorithm == "soundfinder":
 
            for index, row in self.filtered_cross_correlations.iterrows():
                reference = row["reference_file"]
                others = row["other_files"]
                reference_coords = self.aru_coords.loc[reference]
                others_coords = [self.aru_coords.loc[i] for i in others]
                all_coords = [reference_coords] + others_coords
                # add 0 tdoa for reference receiver
                delays = np.insert(row["time_delays"], 0, 0)

                location = soundfinder(all_coords, delays)
                locations.append(location)
            localized["predicted_x"] = [locations[i][0] for i in range(len(locations))]
            localized["predicted_y"] = [locations[i][1] for i in range(len(locations))]
            localized["pseudorange_error"] = [locations[i][2] for i in range(len(locations))]
        else:
            raise UserWarning("Algorithm not recognized")

    def _get_cross_correlations(
        reference_file,
        other_files,
        start_time,
        end_time,
        bandpass_range,
        max_delay,
        SAMPLE_RATE):
        """
        Gets the maximal cross correlations and the time-delay (in s) corresponding to that cross correlation between
        the reference_file and other_files. Setting max_delay ensures that only cross-correlations
        +/- a certain time-delay are returned. i.e if a sound can be a maximum of +/-
        ----
        args:
            reference_file: Path to reference file.
            other_files: List of paths to the other files which will be cross-correlated against reference_file
            start_time: start of time segment (in seconds) to be cross-correlated
            end_time: end of time segment (in seconds) to be cross-correlated.
            bandpass_range: [lower, higher] of bandpass range.
            max_delay: the maximum time (in seconds) to return cross_correlations for. i.e. if the best cross correlation
                        occurs for a time-delay greater than max_delay, the function will not return it, instead it will return
                        the maximal cross correlation within +/- max_delay
            SAMPLE_RATE: the sampling rate of the audio.
        returns:
            ccs: list of maximal cross-correlations for each pair of files.
            time_differences: list of time differences (in seconds) that yield the maximal cross-correlation.
        """
        lower = min(bandpass_range)
        higher = max(bandpass_range)

        reference_audio = Audio.from_file(
            reference_file, offset=start_time, duration=end_time - start_time
        ).bandpass(lower, higher, order=9)
        other_audio = [
            Audio.from_file(i, offset=start_time, duration=end_time - start_time).bandpass(
                lower, higher, order=9
            )
            for i in other_files
        ]

        max_lag = int(
            max_delay * SAMPLE_RATE
        )  # Convert max_delay (in s) to max_lag in samples

        ccs = np.zeros(len(other_audio))
        time_difference = np.zeros(len(other_audio))
        for index, audio_object in enumerate(other_audio):
            ff = reference_audio.samples
            sf = audio_object.samples

            # TODO: Normalize these, so cross-correlation will return values -1<cc<1
            # TODO: verify this makes sense, could there be some floating point issues with this? Is it the right kind
            # of normalization
            ff = ff / np.std(ff)
            sf = sf / np.std(sf)

            cc = correlate(ff, sf, mode="same")  # correlations are per sample
            cc /= min(len(ff), len(sf))
            lags = correlation_lags(ff.size, sf.size, mode="same")

            # slice cc and lags, so we only look at cross_correlations that are between -max_lag and +max_lag
            lower_limit = int(len(cc) / 2 - max_lag)
            upper_limit = int(len(cc) / 2 + max_lag)

            cc = cc[lower_limit:upper_limit]
            lags = lags[lower_limit:upper_limit]

            # from IPython.core.debugger import Pdb; Pdb().set_trace()
            max_cc = np.max(cc)
            lag = -lags[
                np.argmax(cc)
            ]  # in ties (>2 ccs with same max value), argmax returns the first.
            time_difference[index] = lag
            ccs[index] = max_cc
        time_difference = [i / SAMPLE_RATE for i in time_difference]

        return ccs, time_difference
    
    def _get_detections(predictions_df, cnn_score_threshold):
        """
        Takes the predictions_df of CNN scores, chooses only detections > cnn_score_threshold
        and outputs a dictionary of times at which events were detected, and the ARU files they were detected in.
        args:
            predictions_array: a dataframe with multi-index of (file, start, end) with a column that is values for model predictions.
            cnn_score_threshold: the minimum CNN score needed for a time-window to be considered a detection.
        returns:
            A dictionary of predictions, with key (start_time, end_time), and value list of files with detection triggered
            e.g. {(0.0,2.0): [ARU_0.mp3. ARU_1.mp3]}
        """
        # get the detections from the predictions
        # Threshold the scores to above cnn_score_threshold
        booleans = predictions_df.loc[:, :, :] > cnn_score_threshold # find rows above threshold
        indices = booleans[booleans].dropna().index # choose just those rows. dropna required to drop the others
        recorders = indices.get_level_values(0) # get the list of recorders out of the multi-index
        indices = indices.droplevel(level=0) # drop the recorders

        dataframe = pd.DataFrame(data=recorders, index=indices) # df with index (start_time, end_time)
        dataframe = dataframe.sort_index() # done to ensure speed-up and not get performancewarning
        recorders_list = []
        for idx in dataframe.index.unique():
            recorders_in_time = dataframe.loc[idx].values
            recorders_in_time = [i[0] for i in recorders_in_time] # to get recorder path string out of numpy array
            recorders_list.append(recorders_in_time)
        return dict(zip(dataframe.index.unique(), recorders_list))

    def _group_detections(detections, aru_coords, max_distance_between_recorders):
        """
        Takes the detections dictionary and groups detections that are within max_distance_between_recorders of each other.
        args:
            detections: a dictionary of detections, with key (start_time, end_time), and value list of files with detection triggered
            aru_coords: a dictionary of aru coordinates, with key aru file path, and value (x,y) coordinates
            max_distance_between_recorders: the maximum distance between recorders to consider a detection as a single event
        returns:
            A dictionary of grouped detections, with key (start_time, end_time), and value list of files with detection triggered
            e.g. {(0.0,2.0): [ARU_0.mp3. ARU_1.mp3]}
        """
        # group detections that are within max_distance_between_recorders of each other
        # return a dictionary of grouped detections
        # get the coordinates of the recorders
        # get the distance between recorders
        # if the distance is less than max_distance_between_recorders, group the detections
        from itertools import product

        # Group recorders based on being within < max_distance_between_recorders.
        # recorders_in_distance is dictionary in
        # form {ARU_0.mp3: [ARU_1.mp3, ARU_2.mp3...] for all recorders within max_distance_between_recorders }
        recorders_in_distance = dict()

        aru_files = aru_coords_df.index
        for aru in aru_files:  # loop over the aru files
            pos_aru = np.array(aru_coords_df.loc[aru])
            other_arus = np.array(aru_coords_df)
            distances = other_arus - pos_aru
            euclid_distances = [np.linalg.norm(d) for d in distances]

            mask = [
                0 < i <= max_distance_between_recorders for i in euclid_distances
            ]  # boolean mask
            recorders_in_distance[aru] = list(aru_files[mask])

        times = []
        reference_files = []
        other_files = []

        for time_segment in detections.keys():  # iterate through all the time-segments
            for file in detections[
                time_segment
            ]:  # iterate through each file with a call detected in this time-segment
                reference = file  # set this file to be reference
                others = [
                    f for f in detections[time_segment] if f != reference
                ]  # All the other ARUs
                others_in_distance = [
                    aru for aru in others if aru in recorders_in_distance[reference]
                ]  # ARUs close enough

                if (
                    len(others_in_distance) + 1 >= min_number_of_ARUs
                ):  # minimum number of ARUs needed to localize.
                    times.append(time_segment)
                    reference_files.append(reference)
                    other_files.append(others_in_distance)

        grouped_detections = pd.DataFrame(
            data=zip(times, reference_files, other_files),
            columns=["time", "reference_file", "other_files"],
        )
        return grouped_detections

In [33]:
a = Localizer(model= None, files = None, thresholds = None, predictions = None)

In [34]:
# test with a model
from opensoundscape.torch.models.cnn import CNN
import pandas as pd
from pathlib import Path

specky_table = pd.read_csv(Path("woodcock_labeled_data/woodcock_labels.csv"))
from sklearn.model_selection import train_test_split
# create a new dataframe with the filenames from the previous table as the index
labels = pd.DataFrame(index=specky_table['filename'])

#convert 'present' to 1 and 'absent' to 0
labels['woodcock']=[1 if l=='present' else 0 for l in specky_table['woodcock']]

#look at the first rows
labels.head(3)
train_df,validation_df = train_test_split(labels,test_size=0.2,random_state=1)
classes = train_df.columns #in this case, there's just one class: ["woodcock"]

model = CNN('resnet18',classes=classes,sample_duration=0.5)

In [41]:
files = ['./woodcock_labeled_data/' + i for i in labels.index]
a = Localizer(model = model, files = files, thresholds = {"woodcock":2}, predictions = None)

In [44]:
a.get_predictions()

UserWarning: Predictions already exist - set predictions to None if you want to re-run predictions

In [43]:
a.predictions

woodcock
file                                               start_time end_time          
./woodcock_labeled_data/d4c40b6066b489518f8da83... 0.0        0.5      -0.363851
                                                   0.5        1.0       0.333593
                                                   1.0        1.5      -0.242034
                                                   1.5        2.0      -0.339171
./woodcock_labeled_data/e84a4b60a4f2d049d73162e... 0.0        0.5      -0.399152
...                                                                          ...
./woodcock_labeled_data/78654b6f687d7635f50fba3... 1.5        2.0      -0.407226
./woodcock_labeled_data/ec0bd96aee95f03b47628b9... 0.0        0.5       0.599427
                                                   0.5        1.0      -0.290913
                                                   1.0        1.5      -0.384412
                                                   1.5        2.0      -0.289816

[116 rows x 1 columns]

In [37]:
model.predict(files, activation_layer = None)

IndexError: None of the samples in the SafeDataset loaded. All samples caused exceptions during preprocessing. 

In [38]:
files

['d4c40b6066b489518f8da83af1ee4984.wav',
 'e84a4b60a4f2d049d73162ee99a7ead8.wav',
 '79678c979ebb880d5ed6d56f26ba69ff.wav',
 '49890077267b569e142440fa39b3041c.wav',
 '0c453a87185d8c7ce05c5c5ac5d525dc.wav',
 '0fc107ec5e76bf7a98dd207a379ddeb6.wav',
 '50b6b7c7e843597e0dbc69869d10f4e8.wav',
 '35ca80c22127c3c0ae032a08adf8bf11.wav',
 'f87d427bef752f5accbd8990f23c9499.wav',
 '0ab7732b506105717708ea95fb98de63.wav',
 'ad90eefb6196ca83f9cf43b6f56c4b4a.wav',
 'cd0b8d8a89321046e96abee21a5527ca.wav',
 '24073ce519bf1d24107da8a9e126b324.wav',
 '863095c237c52ec51cff7395d70cee41.wav',
 '882de25226ed989b31274eead6630b47.wav',
 '6a83b011665c482c1f260d8e111aa81c.wav',
 '45c4b1ed3d7d0acc27125579c5fcaf5c.wav',
 '4bb7dbc13db479e8b5769dd96be349f4.wav',
 '75b2f63e032dbd6d197900495a16856f.wav',
 '4afa902e823095e03ba23ebc398c35b7.wav',
 '01c5d0c90bd4652f308fd9c73feb1bf5.wav',
 '92647ab903049a9ee4125abdf7b24f2a.wav',
 'c057a4486b25cd638850fc07399385b2.wav',
 'e9e7153d11de3ac8fc3f7164d43bac92.wav',
 '724d8e61b678a6

In [40]:
[Path().resolve() / i for i in files]

[PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/d4c40b6066b489518f8da83af1ee4984.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/e84a4b60a4f2d049d73162ee99a7ead8.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/79678c979ebb880d5ed6d56f26ba69ff.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/49890077267b569e142440fa39b3041c.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/0c453a87185d8c7ce05c5c5ac5d525dc.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/0fc107ec5e76bf7a98dd207a379ddeb6.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/50b6b7c7e843597e0dbc69869d10f4e8.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/35ca80c22127c3c0ae032a08adf8bf11.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/f87d427bef752f5accbd8990f23c9499.wav'),
 PosixPath('/Users/LOF19/Dev/opensoundscape/docs/tutorials/0ab7732b506105717708ea95fb98de63.wav'),
 PosixPath